In [1]:
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import s3fs
import seaborn as sns

sys.path.append("/Users/fgu/dev/projects/mdb_eval")
import src.data.aggregators as agg
import src.data.make_data as md
import src.data.selectors as sl
import src.helpers.data as hd
import src.helpers.io as io

fs = s3fs.S3FileSystem(profile="3di")

sns.set_style("whitegrid")
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [9]:
df = hd.read_txn_data("X11")
hd.inspect(df)

Time for read_txn_data                 : 3.28 minutes
shape: (6,653,551, 35), users: 2734


,date,user_id,amount,desc,merchant,tag_group,tag_spend,user_registration_date,account_created,account_id,account_last_refreshed,account_provider,account_type,birth_year,data_warehouse_date_created,data_warehouse_date_last_updated,id,is_debit,is_female,is_sa_flow,is_salary_pmt,is_urban,latest_balance,lsoa,merchant_business_line,msoa,postcode,region_name,salary_range,tag,tag_auto,tag_manual,tag_up,updated_flag,ym
0,2012-01-03,11,69.750000,david lloyd <mdbremoved>,david lloyd,spend,sports,2010-06-30,1900-01-01,303733,2014-07-24 11:05:00,lloyds,current,1954.0,2014-07-18,2017-08-15,80656,True,0.0,False,False,1.0,150.029999,e01015428,david lloyd,e02003207,bh15 4,south west,10k to 20k,hobbies,gym membership,no tag,gym membership,u,2012-01
1,2012-01-03,11,96.400002,sky digital xxxxxxxxxx9317,sky,spend,"entertainment, tv, media",2010-06-30,1900-01-01,303733,2014-07-24 11:05:00,lloyds,current,1954.0,2014-07-18,2017-08-15,80654,True,0.0,False,False,1.0,150.029999,e01015428,sky,e02003207,bh15 4,south west,10k to 20k,services,"entertainment, tv, media",no tag,media bundle,u,2012-01


Add tt

In [10]:
def add_tt(df):
    df["tt"] = df.ym.view(int) - df.user_registration_date.dt.to_period("m").view(int)
    return df


df = add_tt(df)
hd.inspect(df)

shape: (6,653,551, 36), users: 2734


,date,user_id,amount,desc,merchant,tag_group,tag_spend,user_registration_date,account_created,account_id,account_last_refreshed,account_provider,account_type,birth_year,data_warehouse_date_created,data_warehouse_date_last_updated,id,is_debit,is_female,is_sa_flow,is_salary_pmt,is_urban,latest_balance,lsoa,merchant_business_line,msoa,postcode,region_name,salary_range,tag,tag_auto,tag_manual,tag_up,updated_flag,ym,tt
0,2012-01-03,11,69.750000,david lloyd <mdbremoved>,david lloyd,spend,sports,2010-06-30,1900-01-01,303733,2014-07-24 11:05:00,lloyds,current,1954.0,2014-07-18,2017-08-15,80656,True,0.0,False,False,1.0,150.029999,e01015428,david lloyd,e02003207,bh15 4,south west,10k to 20k,hobbies,gym membership,no tag,gym membership,u,2012-01,19
1,2012-01-03,11,96.400002,sky digital xxxxxxxxxx9317,sky,spend,"entertainment, tv, media",2010-06-30,1900-01-01,303733,2014-07-24 11:05:00,lloyds,current,1954.0,2014-07-18,2017-08-15,80654,True,0.0,False,False,1.0,150.029999,e01015428,sky,e02003207,bh15 4,south west,10k to 20k,services,"entertainment, tv, media",no tag,media bundle,u,2012-01,19


Aggregate to user-month

In [13]:
def agg(df):
    return df.groupby(["user_id", "ym"])["tt"].first().reset_index()


dfa = agg(df)

In [21]:
dfa.tt.value_counts().sort_index()[:100]

-38       1
-37       6
-36     185
-35     196
-34     199
-33     204
-32     210
-31     220
-30     225
-29     234
-28     240
-27     249
-26     253
-25     256
-24     269
-23     279
-22     283
-21     287
-20     295
-19     303
-18     307
-17     317
-16     325
-15     351
-14     358
-13     366
-12    1029
-11    1104
-10    1121
-9     1134
-8     1166
-7     1172
-6     1200
-5     1225
-4     1268
-3     2264
-2     2432
-1     2534
 0     2601
 1     2195
 2     2075
 3     1988
 4     1805
 5     1742
 6     1678
 7     1601
 8     1540
 9     1475
 10    1413
 11    1358
 12    1299
 13    1254
 14    1200
 15    1147
 16    1126
 17    1071
 18    1042
 19     991
 20     954
 21     922
 22     890
 23     862
 24     840
 25     811
 26     786
 27     755
 28     733
 29     707
 30     682
 31     657
 32     647
 33     622
 34     606
 35     596
 36     569
 37     549
 38     528
 39     524
 40     503
 41     475
 42     467
 43     452
 44     432
 45 